In [1]:
import numpy as np
from scipy.stats import *

In [6]:
np.random.seed(42)
N = 10000
bad_cnt = 0
alpha = 0.05

sample_dist = norm(loc=2, scale=3)
mu0 = sample_dist.expect()
for i in range(N):
    test = sample_dist.rvs(10)
    control = sample_dist.rvs(10)
    
    p_value = ttest_ind(test, control, alternative='two-sided').pvalue
    bad_cnt += (p_value < alpha)
bad_cnt / N

0.0539

Мы берем 10000 экспериментов, для каждого прогоняем выборку тест и контрол из 10 элементов, которые распределены нормально и принадлежат нормальному распределению (Т-тест). Получилось больше 5%. Правда ли что критерий некорректен? Посчитаем доверительный интервал Вилсона

In [22]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(count=bad_cnt, nobs=N, alpha=0.05, method='wilson')

(0.04964284138359744, 0.05849976196272022)

Значит истинный FPR лежит в интервале выше. И наче значение лежит в этом интервале)

Теперь заменим нормальное распределение на экспоненциальное и посмотрим как работает Т-тест для маленьких ненормальных выборок:

In [24]:
np.random.seed(42)
N = 10000
bad_cnt = 0
alpha = 0.05

sample_dist = expon(scale=10)
mu0 = sample_dist.expect()
for i in range(N):
    test = sample_dist.rvs(10)
    control = sample_dist.rvs(10)
    
    p_value = ttest_ind(test, control, alternative='two-sided').pvalue
    bad_cnt += (p_value < alpha)
bad_cnt / N

0.0433

In [25]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(count=bad_cnt, nobs=N, alpha=0.05, method='wilson')

(0.03948313833238368, 0.04746760577939231)

Ну и этот критерий не является подходящим, но при этом критерий допускает ошибку реже чем альфа, но это не очень хорошо, потому что мы хотим использовать альфу как что-то конкретное. Это значит что мощность этого алгоритма так себе. Так же напишем функцию для определения, подходит ли нам Т-тест для решения задачи. На вход будет принимать 2 выборки из двух экспоненциальных распределений (одна со сдвигом 5, и сигмой 5, другая с сигмой 5). На выходе будет доверительный интервал

In [30]:
test_dist = expon(scale=10)
control_dist = expon(loc=5, scale=5)
sample_size = 10

In [46]:
def check_criterion(test_dist, control_dist, sample_size, N_exps=N, to_print=True):
    np.random.seed(35)
    bad_cnt = 0
    alpha = 0.05
    
    for i in range(N_exps):
        test = test_dist.rvs(sample_size)
        control = control_dist.rvs(sample_size)
        pvalue = ttest_ind(test, control, equal_var=False, alternative='two-sided').pvalue
        bad_cnt += (pvalue < alpha)
        
    if to_print:
        print(f'FPR: {round(bad_cnt / N_exps, 4)}')
        print(f"CI: {proportion_confint(count=bad_cnt, nobs=N_exps, alpha=0.05, method='wilson')}")
    else:
        return proportion_confint(count=bad_cnt, nobs=N_exps, alpha=0.05, method='wilson')

In [47]:
check_criterion(test_dist=test_dist, control_dist=control_dist, sample_size=10)

FPR: 0.0689
CI: (0.0640994596515586, 0.07403162374363877)


Далее найдем чем отличается большая выборка от маленькой

In [52]:
scale = np.arange(20, 110, 20)
for N in scale:
    left, right = check_criterion(test_dist=test_dist, control_dist=control_dist, sample_size=N, N_exps=10000, to_print=False)
    if left < alpha < right:
        print(f'Sample size = {N}')
        break

Sample size = 60


In [51]:
left, right = check_criterion(test_dist=test_dist, control_dist=control_dist, sample_size=N, N_exps=10000, to_print=False)
print(left, right)

0.0491624202232057 0.057980567121482626


Если подходят 2 критерия то берем тот у которого больше мощномть. Одна выборка пусть будет в 2 раза больше

In [54]:
np.random.seed(42)

N = 10000
rej_cnt = 0
alpha = 0.05

sample_dist = norm(loc=2, scale=3)
mu = sample_dist.expect()

for i in range(N):
    test = sample_dist.rvs(15)
    control = sample_dist.rvs(15) * 2
    
    pvalue = ttest_ind(test, control, equal_var=False, alternative='two-sided').pvalue
    
    rej_cnt += (pvalue < alpha)
rej_cnt / N

0.1938